In [6]:

import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import numpy as np
from hmmlearn import hmm
import itertools
from tqdm import tqdm
import matplotlib.pyplot as plt
from hmmlearn import hmm
from sklearn.preprocessing import StandardScaler

In [7]:
# Ruta al archivo CSV
csv_path = "/home/mazroj/Chaos-Coders/data/coffee_weekly.csv"  # Reemplaza con la ruta a tu archivo CSV
test_size = 0.3  # Porcentaje de datos para el conjunto de prueba

try:
    # Usa pandas para leer el archivo CSV
    used_data = pd.read_csv(csv_path)
    used_data.dropna(inplace=True)
except FileNotFoundError:
    print("Archivo CSV no encontrado. Por favor, verifica la ruta del archivo.")
    sys.exit()
train_data = used_data.copy()

In [3]:
def predictor(train_data, n_components , day_index_1):
    # No mezcles los datos ya que es una serie temporal
    #_train_data, test_data = train_test_split(used_data, test_size=test_size, shuffle=False)

    # Elimina las columnas que no se usan
    train_data = train_data.drop(["Volume", "Adj Close"], axis=1)  # Ajusta los nombres de las columnas según tu dataset
    #test_data = test_data.drop(["Volume", "Adj Close"], axis=1)  # Ajusta los nombres de las columnas según tu dataset

    # Extrae las características: precios de apertura, cierre, máximo y mínimo
    open_price = np.array(train_data["Open"])
    close_price = np.array(train_data["Close"])
    high_price = np.array(train_data["High"])
    low_price = np.array(train_data["Low"])

    # Calcula los cambios fraccionales en los precios alto, bajo y de cierre para usar como conjunto de observaciones
    frac_change = (close_price - open_price) / open_price
    frac_high = (high_price - open_price) / open_price
    frac_low = (open_price - low_price) / open_price

    # Apila las características en columnas
    features = np.column_stack((frac_change, frac_high, frac_low))

    # Inicializa el modelo HMM
    #n_components = 3  # Número de estados en el modelo, ajusta según tus necesidades
    hmm_1 = hmm.GMMHMM(n_components=n_components)

    # Registro de la extracción de características
    #_logger.info(">>> Extracting Features"3
    observations = features  # Usa las características que has calculado
    #_logger.info("Features extraction Completed <<<")

    # Ajusta el modelo HMM usando la función 'fit' de hmmlearn
    hmm_1.fit(observations)

    # Parámetros para los intervalos
    n_intervals_frac_change = 15  # Ajusta según tus necesidades
    n_intervals_frac_high = 15  # Ajusta según tus necesidades
    n_intervals_frac_low = 15  # Ajusta según tus necesidades

    # Crea arrays de NumPy con números espaciados uniformemente para cada rango
    frac_change_range = np.linspace(-0.1, 0.1, n_intervals_frac_change)
    frac_high_range = np.linspace(0, 0.1, n_intervals_frac_high)
    frac_low_range = np.linspace(0, 0.1, n_intervals_frac_low)

    # Calcula todos los resultados posibles utilizando el producto cartesiano
    possible_outcomes = np.array(
        list(itertools.product(frac_change_range, frac_high_range, frac_low_range))
    )

    # Parámetros
    #day_index_1 =   # Índice del día actual, ajusta según tus necesidades
    n_latency_days = 5  # Número de días de latencia, ajusta según tus necesidades

    # Usa los datos anteriores de n_latency_days para las predicciones
    #previous_data_start_index = max(0, day_index_1 - n_latency_days)
    #previous_data_end_index = max(0, day_index_1 - 1)
    previous_data = train_data.iloc[-n_latency_days:]

    # Supongamos que 'features' son las características extraídas
    # features = np.column_stack((frac_change, frac_high, frac_low))  # Descomenta esta línea si necesitas calcular las características
    previous_data_features = features  # Usa las características que has calculado

    outcome_score = []

    # Evalúa todos los resultados posibles y selecciona el más probable para usar en la predicción
    for possible_outcome in possible_outcomes:
        total_data = np.row_stack((previous_data_features, possible_outcome))
        outcome_score.append(hmm_1.score(total_data))

    # Obtén el índice del resultado más probable y devuélvelo
    most_probable_outcome = possible_outcomes[np.argmax(outcome_score)]

    # Obtén el precio de apertura para el día dado
    #open_price = test_data.iloc[day_index_1]["Open"]

    # Supongamos que 'most_probable_outcome' contiene los cambios fraccionales más probables en [frac_change, frac_high, frac_low]
    #predicted_frac_change, pred_frac_high, pred_frac_low = most_probable_outcome

    # Calcula el precio de cierre previsto
    #predicted_close_price = open_price * (1 + predicted_frac_change)

    #print(predicted_close_price)

    # Obtén la fecha correspondiente al day_index
    #date_at_day_index = test_data.iloc[day_index_1]['Date']  # Reemplaza 'Fecha' con el nombre de tu columna de fechas
    #print(date_at_day_index)




    return most_probable_outcome

In [4]:
# Supongamos que 'days' es el número total de días en tus datos de prueba
days = 1

# Inicializa una lista vacía para almacenar los precios de cierre previstos
predicted_close_prices = []
predicted_change_list = []

# Imprime el rango de fechas para las que se harán las predicciones
#print(
#    "Predicting Close prices from "
#    + str(train_data.index[0])
##    + " to "
 #   + str(test_data.index[-1])
#)

# Bucle para predecir los precios de cierre para cada día en el período de prueba
for day_index in tqdm(range(days)):
    # Aquí, puedes usar el código que escribimos anteriormente para predecir el precio de cierre para un día dado
    open_price = train_data.iloc[-1]["Close"]
    predicted_frac_change, pred_frac_high, pred_frac_low = predictor(train_data,3, day_index)  # Usa el resultado más probable que calculaste previamente
    predicted_close_price = open_price * (1 + predicted_frac_change)
    
    # Añade el precio de cierre previsto a la lista
    predicted_close_prices.append(predicted_close_price)
    predicted_change_list.append(predicted_frac_change)

# Almacena la lista de precios de cierre previstos en una variable
predicted_close = predicted_close_prices

100%|██████████| 1/1 [00:12<00:00, 12.98s/it]


In [5]:
predicted_close

[165.649994]

In [6]:
predicted_frac_change

0.0

In [8]:
# Function to predict the next day's closing price based on the last n_latency_days in the dataset

_train_data, test_data = train_test_split(used_data, test_size=test_size, shuffle=False)
n_latency_days = 5
train_data = train_data.drop(["Volume", "Adj Close"], axis=1) 
#Ajusta los nombres de las columnas según tu dataset
test_data = test_data.drop(["Volume", "Adj Close"], axis=1)  # Ajusta los nombres de las columnas según tu dataset

# Extrae las características: precios de apertura, cierre, máximo y mínimo
open_price = np.array(train_data["Open"])
close_price = np.array(train_data["Close"])
high_price = np.array(train_data["High"])
low_price = np.array(train_data["Low"])

# Calcula los cambios fraccionales en los precios alto, bajo y de cierre para usar como conjunto de observaciones
frac_change = (close_price - open_price) / open_price
frac_high = (high_price - open_price) / open_price
frac_low = (open_price - low_price) / open_price

# Apila las características en columnas
features = np.column_stack((frac_change, frac_high, frac_low))

# Inicializa el modelo HMM
#n_components = 3  # Número de estados en el modelo, ajusta según tus necesidades
hmm_1 = hmm.GMMHMM(n_components=3)

# Registro de la extracción de características
#_logger.info(">>> Extracting Features"3
observations = features  # Usa las características que has calculado
#_logger.info("Features extraction Completed <<<")

# Ajusta el modelo HMM usando la función 'fit' de hmmlearn
hmm_1.fit(observations)

# Parámetros para los intervalos
n_intervals_frac_change = 15  # Ajusta según tus necesidades
n_intervals_frac_high = 15  # Ajusta según tus necesidades
n_intervals_frac_low = 15  # Ajusta según tus necesidades

# Crea arrays de NumPy con números espaciados uniformemente para cada rango
frac_change_range = np.linspace(-0.1, 0.1, n_intervals_frac_change)
frac_high_range = np.linspace(0, 0.1, n_intervals_frac_high)
frac_low_range = np.linspace(0, 0.1, n_intervals_frac_low)

# Calcula todos los resultados posibles utilizando el producto cartesiano
possible_outcomes = np.array(list(itertools.product(frac_change_range, frac_high_range, frac_low_range)))

def predict_next_day():
    # Use the last n_latency_days from the test set for the prediction
    last_days_data = test_data.iloc[-n_latency_days:]

    outcome_score = []

    for possible_outcome in possible_outcomes:
        total_data = np.row_stack((features[-n_latency_days:], possible_outcome))
        outcome_score.append(hmm_1.score(total_data))

    most_probable_outcome = possible_outcomes[np.argmax(outcome_score)]

    last_open_price = last_days_data.iloc[-1]["Open"]
    predicted_frac_change, _, _ = most_probable_outcome
    predicted_close_price = last_open_price * (1 + predicted_frac_change)

    return predicted_close_price, predicted_frac_change


In [9]:
# Predict the next day's closing price
next_day_predicted_price = predict_next_day()
print(f'Predicted closing price for the next day: {next_day_predicted_price}')

Predicted closing price for the next day: (163.399994, 0.0)
